In [ ]:
import os

# ... Hide tensorflow debug messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# ... Disable GPU
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

import tensorflow as tf

from utils import (
    DataLoader,
    PerformanceMetrics,
    get_subject_ids,
    perform_loocv,
    reset_weights
)
from models import NDDNet


# Config

In [ ]:
DATA_DIR = "../data/dataset/"

FS = 300
N_EPOCHS = 300
LEARNING_RATE = 3e-4
GRF_SEGMENT_LEN = 120
VAL_PERCENTAGE = 0.3
MODE = "combined"

# Utils

In [ ]:
all_subject_ids = get_subject_ids(DATA_DIR, "all")
dataloader = DataLoader(DATA_DIR, all_subject_ids)
metrics = PerformanceMetrics()


# Model

In [7]:
network = NDDNet(
    n_classes=2,
    n_conv_blocks=5,
    n_mlp_layers=1,
    kernel_size=5,
    conv_channel_width=16,
    mlp_channel_width=32,
    mode="combined"
)

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)


# ALS vs HC

In [8]:
DISEASE = "als"

metrics = PerformanceMetrics()
subject_ids = get_subject_ids(DATA_DIR, DISEASE)

for test_subject in subject_ids:
    print(f"Training on {test_subject} ... ")
    y_true, y_pred, model = perform_loocv(
        test_subject=test_subject,
        val_percentage=VAL_PERCENTAGE,
        subject_ids=subject_ids,
        dataloader=dataloader,
        network=network,
        loss=loss,
        optimizer=optimizer,
        n_epochs=N_EPOCHS
    )

    metrics.add_entry(test_subject, y_true, y_pred)

    break

metrics.print_metrics()



Training on als1 ... 


ValueError: in user code:

    File "/home/ai/Ai/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/ai/Ai/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ai/Ai/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/ai/Ai/lib/python3.10/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/ai/Ai/lib/python3.10/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "/home/ai/Ai/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/home/ai/Ai/li